### Trading Testing

Testing delta probability for weeklies and monthlies
    - Does delta match probability of occurrence
    - Check Kelly criterion for long straddles/delta neutral back ratios
        - To calculate expected losses and gains, use options pricing module to interpolate prices at given percentage
        moves and use the normal pdf as the probability weightings
    
Check earnings returns post announcement factors
    - Factors:
        - Number of times beaten earnings (Dummy Variable)
        - Consecutive earnings beats
        - Consecutive earnings upsets
        - 3 Month Trend before earnings
        - YTD Trend
        - Momentum of monthly returns (20 day, 60 day)


In [1]:
import datetime as dt

import numpy as np
import pandas as pd
from pandas_datareader.data import Options
from py_vollib.black_scholes_merton.implied_volatility import *
# ts = TimeSeries(key='5HZEUI5AFJB06BUK',output_format='pandas')
import py_vollib
from py_vollib.black_scholes_merton.implied_volatility import *
from py_vollib.black_scholes_merton.greeks.analytical import *
import plotly
import os
import pandas_market_calendars as mcal
import json
from pandas.io.json import json_normalize
import urllib.request as req
import time
nyse = mcal.get_calendar('NYSE')
from helpers import *

from scipy.stats import norm as norm
# '''
# functions list:
    
#     maturities(dt.datetime()) --> [float(front_wgt), float(back_wgt)]
    
#     optionslam_scrape(str[ticker]) --> dict[earnings]
    
#     yahoo_table_parse(str[raw_html_table]) --> DataFrame[ticker]
    
#     yahoo_earnings(dt.datetime()) --> DataFrame[earnings_on_date]
    
#     fundamentals(str[ticker]) --> DataFrame[stock_fundamentals]
    
#     get_fundas(list[ticker_lst]) --> DataFrame[stock_fundamentals]
    
#     historical_data(str[ticker], int[day_number], int[rolling_window], outsize[str]) --> DataFrame[daily_stock_data]
    
#     current_volatility(list[ticker_lst], int[roll]) --> DataFrame[stock_volatilities]
    
#     all_options(str[ticker], bool[greeks]) --> DataFrame[options_chains]
    
#     earnings_condor(str[ticker], int[max_gap], int[dte_thresh], float[|money_thresh| <= 1]) -- DataFrame[condors], DataFrame[puts], DataFrame[calls]
    
#     write_excel(str[filename], list[str[sheetnames]], list[dataframes]) --> void()
    
#     curr_stock_data(str[ticker]) --> DataFrame[stock_info]
    
#     curr_batch_quotes(list_of_string[tickers]) --> DataFrame[stock_info]

#     past_earnings(str[ticker]) --> DataFrame[earnings_info]

#     earnings_history(str[ticker]) --> [DataFrame[earnings_estimate], DataFrame[past_earnings], DataFrame[earnings_estimate_change]]
    
#     av_data(str[ticker]) --> DataFrame[ticker_open, ticker_close]

#     av_batch(list_of_str[tickers]) --> DataFrame[tickers_closes]

#     check_mkt_corr(int[corr_rolling_window],int[plot_window]) --> DataFrame[rolling_corr]

#     vvix_check() --> DataFrame[VVIX Data]

#     earnings_longs(list_of_str[ticker], float[bid_ask_spread]) --> DataFrame[option_chains]

#     all_options_v2(str[ticker], int[dte_ub], int[dte_lb], float[moneyness]) --> DataFrame[option_chains]

#     yahoo_options_query(str[ticker], int[dte_ub], int[dte_lb]) --> DataFrame[option_chains]

#     greek_calc(DataFrame[option_chain], str[prem_price_use], str[day_format], float[interest_rate], float[dividend_rate])

#     price_sim(DataFrame[options_df], float[price_change], float[vol_change], int[days_change], str[output = 'All'],
#               str[skew = 'flat'], str[day_format = 'trading'], float[interest_rate = 0.0193], float[dividend_rate = 0],
#               float[prem_price_use = 'Mid'])


#     position_sim(DataFrame[position_df], list_of_int[holdings], int[shares],
#                  float[price_change], float[vol_change], int[dte_change], str[output = 'All'],
#                  str[skew = 'flat'], str[prem_price_use = 'Mid'], str[day_format = 'trading'], 
#                  float[interest_rate = 0.0193], float[dividend_rate = 0])

#     yahoo_fundamentals(list_of_str[tickers]) --> DataFrame[fundamentals]

# '''

In [2]:
# Pulling Indices Data

spx = pd.read_csv('spx.csv', index_col = 0)[['Open','Close']]
spx.columns = ['spx_{0}'.format(x) for x in spx.columns]

vix = pd.read_csv('vix.csv', index_col = 0)[['Open','Close']]
vix.columns = ['vix_{0}'.format(x) for x in vix.columns]

vvix = pd.read_csv('vvix.csv', index_col = 0)[['Close']]
vvix.columns = ['vvix']

yields = pd.read_csv('https://www.quandl.com/api/v3/datasets/USTREASURY/YIELD.csv?api_key=dzmzEExntfap7SNx5p6t', index_col = 0)
df = pd.concat([spx,yields[yields.columns[1:]]], axis = 1).dropna()
df.index = pd.to_datetime(df.index)
df[df.columns] = df[df.columns].apply(pd.to_numeric, errors='coerce')

In [3]:
# Creating full calendar

nyse_dates = nyse.schedule(start_date=df.index[0].date().strftime('%Y-%m-%d'), end_date=df.index[-1].date().strftime('%Y-%m-%d'))
date_idx = [x.date() for x in mcal.date_range(nyse_dates, frequency='1D')]

date_df = pd.DataFrame(columns = ['Trading_Day'],
                       index = range(len(date_idx)))
date_df.index = date_idx
date_df.index = pd.to_datetime(date_df.index)
date_df['Trading_Day'] = True

start_date = dt.datetime(1993,12,1).date()

full_cal = []
curr_date = start_date
while curr_date != dt.datetime.today().date():
    full_cal.append(curr_date)
    curr_date = curr_date + dt.timedelta(days = 1)
    
calendar_df = pd.DataFrame(columns = ['Weekday'],
                           index = full_cal)
calendar_df.index = pd.to_datetime(calendar_df.index)
calendar_df['Weekday'] = calendar_df.index.weekday
date_df = calendar_df.join(date_df).fillna(False)

In [4]:
spx_check = date_df.join(df)
spx_check = spx_check[spx_check['Weekday'].isin([0,1,2,3,4])]
spx_check['Week'] = spx_check.index.week
spx_check['Month'] = spx_check.index.month
spx_check['Year'] = spx_check.index.year
spx_check['Month_Expiry'] = False
spx_check['New_Month'] = False
spx_check['Week_Start'] = False
spx_check['Week_End'] = False

# Marking Monthly Expirations for SPX and Following Trading Day
for idx, row in spx_check[['Month','Year']].drop_duplicates().iterrows():
    curr_eigth_day = dt.date(row['Year'],row['Month'],7)
    curr_third_fri = curr_eigth_day - dt.timedelta(dt.date(row['Year'],row['Month'],3).weekday()) + dt.timedelta(14)
    curr_third_thurs = curr_eigth_day - dt.timedelta(dt.date(row['Year'],row['Month'],4).weekday()) + dt.timedelta(14)
    curr_third_wed = curr_eigth_day - dt.timedelta(dt.date(row['Year'],row['Month'],5).weekday()) + dt.timedelta(14)
    
    # Marking monthly expiry days: Third Friday of every month, otherwise third thursday
    if spx_check.loc[curr_third_fri,'spx_Open'] != np.nan:
        spx_check.loc[curr_third_fri,'Month_Expiry'] = True
        
    elif spx_check.loc[curr_third_thurs,'spx_Open'] != np.nan:
        spx_check.loc[curr_third_thurs,'Month_Expiry'] = True
        
    elif spx_check.loc[curr_third_wed,'spx_Open'] != np.nan:
        spx_check.loc[curr_third_wed,'Month_Expiry'] = True
        
# Marking Monthly Starting
for idx, row in spx_check[spx_check['Month_Expiry'] == True].iterrows():
    following_week = spx_check[spx_check.index > idx].head()
    for idx_2, row_2 in following_week.iterrows():
        if row_2['Trading_Day']:
            spx_check.loc[idx_2, 'New_Month'] = True
            break
            
# Setting Weekly Tags

for idx, row in spx_check[['Week','Year']].drop_duplicates().iterrows():
    current_week = spx_check[(spx_check['Week'] == row['Week']) &
                             (spx_check['Year'] == row['Year']) &
                             (spx_check['Trading_Day'] == True)]
    week_start = current_week.head(1)
    week_end = current_week.tail(1)
    
    try:
        spx_check.loc[week_start.index[0].date(), 'Week_Start'] = True
        spx_check.loc[week_end.index[0].date(), 'Week_End'] = True
    except:
        continue

spx_check['Monthlies'] = spx_check['Month_Expiry'] | spx_check['New_Month']
spx_monthly_check = spx_check[spx_check['Monthlies'] == True]

spx_check['Weeklies'] = spx_check['Week_Start'] | spx_check['Week_End']
spx_weekly_check = spx_check[(spx_check['Weeklies'] == True) &
                             (spx_check['Year'] > 1993) &
                             (spx_check['Year'] < 2018)]

# Filling in missing values
spx_monthly_check.spx_Open = spx_monthly_check.spx_Open.fillna((spx_monthly_check.spx_Open.shift() + 
                                                                spx_monthly_check.spx_Open.shift(-1))/2)
spx_monthly_check.spx_Close = spx_monthly_check.spx_Close.fillna((spx_monthly_check.spx_Close.shift() + 
                                                                  spx_monthly_check.spx_Close.shift(-1))/2)

spx_weekly_check.spx_Open = spx_weekly_check.spx_Open.fillna((spx_weekly_check.spx_Open.shift() + 
                                                              spx_weekly_check.spx_Open.shift(-1))/2)
spx_weekly_check.spx_Close = spx_weekly_check.spx_Close.fillna((spx_weekly_check.spx_Close.shift() + 
                                                                spx_weekly_check.spx_Close.shift(-1))/2)

C:\Users\Fang\Anaconda3\lib\site-packages\pandas\core\generic.py:2999: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [5]:
spx_weekly_check = spx_weekly_check[['spx_Open','spx_Close','Week_Start','Week_End','Year'] + list(yields.columns[1:])]
spx_weekly_check['Week_Return'] = spx_weekly_check.spx_Close/spx_weekly_check.spx_Open.shift(1) - 1
spx_weekly_check = spx_weekly_check[spx_weekly_check['Week_End'] == True]
spx_weekly_check['Up_Moves'] = (spx_weekly_check['Week_Return'] > 0.005)*1
spx_weekly_check['Down_Moves'] = (spx_weekly_check['Week_Return'] < -0.005)*1
spx_weekly_check['No_Moves'] = 1 - spx_weekly_check['Up_Moves'] - spx_weekly_check['Down_Moves']

weekly_counts = {'Ups':[],'Downs':[],'Flats':[]}
weekly_tally = spx_weekly_check[['Up_Moves','Down_Moves', 'No_Moves']]

for yr in spx_weekly_check.Year.drop_duplicates():
    weekly_counts['Ups'].append(weekly_tally[weekly_tally.index.year == yr].sum()['Up_Moves'])
    weekly_counts['Downs'].append(weekly_tally[weekly_tally.index.year == yr].sum()['Down_Moves'])
    weekly_counts['Flats'].append(weekly_tally[weekly_tally.index.year == yr].sum()['No_Moves'])
    
weekly_df = pd.DataFrame(weekly_counts, index = spx_weekly_check.Year.drop_duplicates())
weekly_df = weekly_df.drop([2002,2003,2004,2005,2006])
weekly_df_pct = pd.concat([(weekly_df['Ups']/weekly_df.sum(axis = 1)),
                           (weekly_df['Downs']/weekly_df.sum(axis = 1)),
                           (weekly_df['Flats']/weekly_df.sum(axis = 1))],axis = 1)
weekly_df_pct.columns = ['UpMove','DownMove','NoMove']
weekly_df_pct

,UpMove,DownMove,NoMove
Year,,,
1994,0.365385,0.288462,0.346154
1995,0.576923,0.173077,0.250000
1996,0.519231,0.326923,0.153846
1997,0.461538,0.365385,0.173077
1998,0.547170,0.320755,0.132075
1999,0.519231,0.365385,0.115385
2000,0.423077,0.538462,0.038462
2001,0.442308,0.442308,0.115385
2007,0.480769,0.346154,0.173077


In [6]:
spx_monthly_check = spx_monthly_check[['spx_Open','spx_Close','New_Month','Month_Expiry','Year'] + list(yields.columns[1:])]
spx_monthly_check['Month_Return'] = spx_monthly_check.spx_Close/spx_monthly_check.spx_Open.shift(1) - 1
spx_monthly_check = spx_monthly_check[spx_monthly_check['Month_Expiry'] == True]
spx_monthly_check['Up_Moves'] = (spx_monthly_check['Month_Return'] > 0.01)*1
spx_monthly_check['Down_Moves'] = (spx_monthly_check['Month_Return'] < -0.01)*1
spx_monthly_check['No_Moves'] = 1 - spx_monthly_check['Up_Moves'] - spx_monthly_check['Down_Moves']

monthly_counts = {'Ups':[],'Downs':[],'Flats':[]}
monthly_tally = spx_monthly_check[['Up_Moves','Down_Moves', 'No_Moves']]

for yr in spx_monthly_check.Year.drop_duplicates():
    monthly_counts['Ups'].append(monthly_tally[monthly_tally.index.year == yr].sum()['Up_Moves'])
    monthly_counts['Downs'].append(monthly_tally[monthly_tally.index.year == yr].sum()['Down_Moves'])
    monthly_counts['Flats'].append(monthly_tally[monthly_tally.index.year == yr].sum()['No_Moves'])
    
monthly_df = pd.DataFrame(monthly_counts, index = spx_monthly_check.Year.drop_duplicates())
monthly_df = monthly_df.drop([2002,2003,2004,2005,2006])
monthly_df_pct = pd.concat([(monthly_df['Ups']/monthly_df.sum(axis = 1)),
                           (monthly_df['Downs']/monthly_df.sum(axis = 1)),
                           (monthly_df['Flats']/monthly_df.sum(axis = 1))],axis = 1)
monthly_df_pct.columns = ['UpMove','DownMove','NoMove']
monthly_df_pct

,UpMove,DownMove,NoMove
Year,,,
1993,0.000000,0.000000,1.000000
1994,0.333333,0.250000,0.416667
1995,0.916667,0.000000,0.083333
1996,0.583333,0.166667,0.250000
1997,0.583333,0.333333,0.083333
1998,0.666667,0.250000,0.083333
1999,0.416667,0.166667,0.416667
2000,0.333333,0.583333,0.083333
2001,0.416667,0.416667,0.166667


In [7]:
monthly_df

,Downs,Flats,Ups
Year,,,
1993,0,1,0
1994,3,5,4
1995,0,1,11
1996,2,3,7
1997,4,1,7
1998,3,1,8
1999,2,5,5
2000,7,1,4
2001,5,2,5


In [150]:
contracts = spx_options.loc[[49,17]]
holding_days = 2
positions = [1, 1]
day_format = 'trading'
prem_price_use = 'Mid'

def ev_calc(contracts, positions, holding_days, price_stds = 5, day_format = 'trading', prem_price = 'Mid'):
    vix = curr_stock_data('^VIX')[-1]/100

    if day_format != 'trading':
        year = 365
    else:
        year = 252

    def kelly(odds, p, q):
        return (odds*p - q)/odds

    period_std = vix/np.sqrt(year/holding_days)

    returns = np.linspace(-price_stds*period_std,price_stds*period_std, 300)
    returns = pd.DataFrame({'Returns': returns,
                            'CDF': norm.cdf(returns, 0, period_std)}, index = range(len(returns)))

    returns['probs'] = round(returns['CDF'] - returns['CDF'].shift(1), 6).fillna(0)
    sims = []
    for idx, row in returns.iterrows():
        sims.append(position_sim(contracts, positions, 0, row.Returns, 0, holding_days, prem_price_use = prem_price))

    sim_df = pd.concat(sims,axis = 0)
    sim_df.index = range(len(returns))
    sim_df = sim_df.join(returns)[['PnL','probs','Percent Return']]

    sim_df['EV'] = sim_df.probs*sim_df.PnL
    expected_val = sim_df.EV.sum()
    expected_gain = sim_df[sim_df['EV'] >= 0].EV.sum()
    expected_loss = sim_df[sim_df['EV'] < 0].EV.sum()
    gain_prob = sim_df[sim_df['EV'] >= 0].probs.sum()
    loss_prob = sim_df[sim_df['EV'] < 0].probs.sum()

    out_df = pd.DataFrame({'EV': expected_val,
                           'Expected Win': expected_gain,
                           'Prob Win': gain_prob,
                           'Expected Loss': expected_loss,
                           'Prob Loss': loss_prob,
                           'Net Cost': (contracts.Mid*contracts.Pos).sum()*100,
                           'Kelly Fraction': kelly(expected_gain/((contracts.Mid*contracts.Pos).sum()*100), gain_prob, loss_prob)},
                          index = [0])
    
    return out_df

In [2]:
raw_options = yahoo_options_query('^SPX', 40, 20)
raw_options.columns = ['Strike', 'Type', 'Expiry', 'DTE',
                       'inTheMoney', 'Ask', 'Bid', 'Mid',
                       'Last', 'IV', 'Underlying_Price']
raw_options = raw_options.replace('c','call')
raw_options = raw_options.replace('p','put')
raw_options

2018-09-20 17:00:00
2018-09-27 17:00:00


,Strike,Type,Expiry,DTE,inTheMoney,Ask,Bid,Mid,Last,IV,Underlying_Price
0,500.0,call,2018-09-27,30,True,2396.0,2394.2,2395.10,2394.40,0.000010,2893.365
1,600.0,call,2018-09-27,30,True,2196.2,2179.3,2187.75,2015.40,0.000010,2893.365
2,800.0,call,2018-09-27,30,True,1995.7,1989.3,1992.50,1993.29,0.000010,2893.365
3,900.0,call,2018-09-27,30,True,1891.7,1884.5,1888.10,1848.20,0.000010,2893.365
4,1000.0,call,2018-09-27,30,True,1798.0,1780.9,1789.45,1786.90,0.000010,2893.365
5,1100.0,call,2018-09-27,30,True,1692.6,1685.5,1689.05,1660.00,0.000010,2893.365
6,1200.0,call,2018-09-27,30,True,1593.3,1586.2,1589.75,1561.35,0.000010,2893.365
7,1250.0,call,2018-09-27,30,True,1543.3,1536.2,1539.75,1528.80,0.000010,2893.365
8,1300.0,call,2018-09-27,30,True,1534.5,1531.9,1533.20,1553.13,0.000010,2893.365
9,1400.0,call,2018-09-27,30,True,1475.1,1473.1,1474.10,1472.93,0.000010,2893.365


In [3]:
import time
import warnings
warnings.filterwarnings('ignore')

start_time = time.time()

raw = raw_options.copy()
greeks = all_greeks(raw)

end_time = time.time()

print("Runtime: {}".format(end_time - start_time))

greeks

Runtime: 0.48363780975341797


,Strike,Type,Expiry,DTE,inTheMoney,Ask,Bid,Mid,Last,IV,Underlying_Price,Rho,Vega,Gamma,Theta,Delta
0,500.0,call,2018-09-27,30,True,2396.0,2394.2,2395.10,2394.40,0.000010,2893.365,0.593872,0.000000,0.000000,-0.026378,1.000000
1,600.0,call,2018-09-27,30,True,2196.2,2179.3,2187.75,2015.40,0.000010,2893.365,0.712646,0.000000,0.000000,-0.031653,1.000000
2,800.0,call,2018-09-27,30,True,1995.7,1989.3,1992.50,1993.29,0.000010,2893.365,0.950195,0.000000,0.000000,-0.042204,1.000000
3,900.0,call,2018-09-27,30,True,1891.7,1884.5,1888.10,1848.20,0.000010,2893.365,1.068970,0.000000,0.000000,-0.047480,1.000000
4,1000.0,call,2018-09-27,30,True,1798.0,1780.9,1789.45,1786.90,0.000010,2893.365,1.187744,0.000000,0.000000,-0.052755,1.000000
5,1100.0,call,2018-09-27,30,True,1692.6,1685.5,1689.05,1660.00,0.000010,2893.365,1.306518,0.000000,0.000000,-0.058031,1.000000
6,1200.0,call,2018-09-27,30,True,1593.3,1586.2,1589.75,1561.35,0.000010,2893.365,1.425293,0.000000,0.000000,-0.063306,1.000000
7,1250.0,call,2018-09-27,30,True,1543.3,1536.2,1539.75,1528.80,0.000010,2893.365,1.484680,0.000000,0.000000,-0.065944,1.000000
8,1300.0,call,2018-09-27,30,True,1534.5,1531.9,1533.20,1553.13,0.000010,2893.365,1.544067,0.000000,0.000000,-0.068582,1.000000
9,1400.0,call,2018-09-27,30,True,1475.1,1473.1,1474.10,1472.93,0.000010,2893.365,1.662842,0.000000,0.000000,-0.073858,1.000000


In [4]:
def position_sim(position_df, holdings, shares,
                 price_change, vol_change, dte_change, output = 'All',
                 skew = 'flat', prem_price_use = 'Mid', day_format = 'trading', 
                 interest_rate = 0.0193, dividend_rate = 0):
    
    if prem_price_use != 'Mid':
        price_col = 'Last'
    else:
        price_col = 'Mid'
                
    position = position_df[['Strike','DTE','Type','IV','Underlying_Price',price_col]]
    position['Pos'] = holdings
    position_dict = {}
    position_dict['Total Cost'] = sum(position[price_col]*position['Pos'])*100 + shares*position['Underlying_Price'].values[0]
    
    simulation = price_sim(position, price_change, vol_change, dte_change, output,
                           skew, day_format, interest_rate, dividend_rate,
                           prem_price_use)
    
    position_dict['Simulated Price'] = sum(simulation['Simulated Prices']*position['Pos'])*100 + shares*position['Underlying_Price'].values[0]*(1 + price_change)
    position_dict['PnL'] = position_dict['Simulated Price'] - position_dict['Total Cost']
    
    if position_dict['Total Cost'] > 0:
        position_dict['Percent Return'] = position_dict['PnL']/position_dict['Total Cost']
    else:
        position_dict['Percent Return'] = -position_dict['PnL']/position_dict['Total Cost']
    
    position_dict['Simulated Delta'] = sum(simulation['Delta']*position['Pos']) + shares/100
    position_dict['Simulated Gamma'] = sum(simulation['Gamma']*position['Pos'])
    position_dict['Simulated Theta'] = sum(simulation['Theta']*position['Pos'])
    position_dict['Simulated Vega'] = sum(simulation['Vega']*position['Pos'])
    position_dict['Simulated Rho'] = sum(simulation['Rho']*position['Pos'])
    
    outframe = pd.DataFrame(position_dict, index = [vol_change])
    return outframe

In [13]:
start_time = time.time()

print(position_sim(position_df, [3, -1], 0, -0.01, 0.01, 20))


end_time = time.time()
print("Runtime: {}".format(end_time - start_time))

      Percent Return  PnL  Simulated Delta  Simulated Gamma  Simulated Price  \
0.01             NaN  NaN              NaN              NaN              NaN   

      Simulated Rho  Simulated Theta  Simulated Vega  Total Cost  
0.01            NaN              NaN             NaN      5305.0  
Runtime: 0.2091531753540039


In [54]:
position_df = greeks.loc[[675,678]]
position_df

,Strike,Type,Expiry,DTE,inTheMoney,Ask,Bid,Mid,Last,IV,Underlying_Price,Rho,Vega,Gamma,Theta,Delta
675,2825.0,put,2018-09-27,30,False,12.5,12.2,12.35,12.51,0.115266,2893.365,-0.876992,3.162926,0.002753,-0.380562,-0.248600
678,2845.0,put,2018-09-27,30,False,15.5,15.1,15.30,14.94,0.107263,2893.365,-1.043705,3.449623,0.003227,-0.379415,-0.295955


In [55]:
price_change = [-0.03, 0.03]
dte_change = [0, 30]
spacing = 10
holdings = [3, -1]

vol_change = [-0.01, 0.03]
vol_spacing = 2

price_col = 'Mid'
output = 'All'
shares = 30
skew = 'flat'

day_format = 'trading'
interest_rate = 0.0193
dividend_rate = 0
prem_price_use = 'Mid'

price_changes = np.linspace(price_change[0], price_change[-1], spacing)
dte_changes = np.linspace(dte_change[0], dte_change[-1], dte_change[-1] - dte_change[0] + 1)

if vol_spacing <= 2:
    vol_changes = vol_change
else:
    vol_changes = np.linspace(vol_change[0], vol_change[-1], vol_spacing)

In [53]:
start_time = time.time()

position = position_df.reset_index()[['Strike','Expiry','DTE','Type','IV','Underlying_Price',price_col]]
position['Pos'] = holdings
initial_cost = sum(position[price_col]*position['Pos'])*100 + shares*position['Underlying_Price'].values[0]

adj_dfs = []

for vol_change in vol_changes:
    # mesh_shape = np.meshgrid(price_changes,dte_changes)
    price_ax, dte_ax = np.meshgrid(price_changes,dte_changes)
    
    vol_adj_df = pd.DataFrame(np.array(np.meshgrid(price_changes,dte_changes)).reshape(2,-1).T)
    vol_adj_df.columns = ['ret_change', 'dte_change']

    indi_sims = []
    for idx, row in position.iterrows():
        curr_sim = pd.DataFrame(index = range(len(vol_adj_df)))
        curr_sim['Strike'] = row.Strike
        curr_sim['DTE'] = row.DTE - vol_adj_df['dte_change']
        curr_sim[curr_sim['DTE'] < 0] = 0

        curr_sim['Type'] = row.Type
        curr_sim['IV'] = row.IV
        curr_sim['Underlying_Price'] = (1 + vol_adj_df[['ret_change']])*row.Underlying_Price
        curr_sim = price_sim(curr_sim, 0, vol_change, 0, output,
                             skew, day_format, interest_rate, dividend_rate,
                             prem_price_use)
        indi_sims.append(curr_sim)
        
    if len(holdings) < 2:
        try:
            adj_df = indi_sims[0]
        except:
            break
        adj_df['Delta'] = adj_df['Delta'] + shares/100
        adj_df['PnL'] = position.head(1)['Pos'][0]*(adj_df['Simulated Prices'] - 
                                                    position.head(1)[price_col][0])*100 + shares*(adj_df['Underlying_Price'] - 
                                                                                                  position.head(1)['Underlying_Price'][0])
        adj_df['Percent Return'] = adj_df['PnL']/initial_cost
    else:
        adj_df = curr_sim[['DTE','Underlying_Price']]
        adj_df['Delta'] = 0
        adj_df['Gamma'] = 0
        adj_df['Vega'] = 0
        adj_df['Theta'] = 0
        adj_df['Rho'] = 0
        adj_df['PnL'] = 0
        for i, val in enumerate(holdings):
            adj_df['Delta'] = adj_df['Delta'] + val*indi_sims[i]['Delta']
            adj_df['Gamma'] = adj_df['Gamma'] + val*indi_sims[i]['Gamma']
            adj_df['Vega'] = adj_df['Vega'] + val*indi_sims[i]['Vega']
            adj_df['Theta'] = adj_df['Theta'] + val*indi_sims[i]['Theta']
            adj_df['Rho'] = adj_df['Rho'] + val*indi_sims[i]['Rho']
            adj_df['PnL'] = adj_df['PnL'] + val*indi_sims[i]['Simulated Prices']
        
        adj_df['PnL'] = (adj_df['PnL'] - sum(position[price_col]*position['Pos']))*100 - shares*(adj_df['Underlying_Price'] -
                                                                                                 position.head(1)['Underlying_Price'][0])
        adj_df['Percent Return'] = adj_df['PnL']/initial_cost
    adj_dfs.append(adj_df)
    
    # adj_dfs.append(indi_sims)
    
end_time = time.time()
print("Runtime: {}".format(end_time - start_time))

ValueError: Length of values does not match length of index

In [44]:
# np.array(np.meshgrid(price_changes,dte_changes)).shape

# np.reshape(np.ravel(np.array(np.meshgrid(price_changes,dte_changes)).reshape(2,-1).T), 
#            np.array(np.meshgrid(price_changes,dte_changes)).shape)
position

,Strike,Expiry,DTE,Type,IV,Underlying_Price,Mid,Pos
0,2825.0,2018-09-27,30,put,0.115266,2893.365,12.35,1


In [45]:
adj_dfs[0]

,Strike,DTE,Type,IV,Underlying_Price,Rho,Vega,Theta,Gamma,Delta,Simulated Prices,PnL,Percent Return
0,2825.0,30.0,put,0.105266,2806.56405,-1.858076,3.844356,-0.383131,0.003895,-0.239371,47.005510,861.522539,0.009786
1,2825.0,30.0,put,0.105266,2980.16595,-0.216743,1.227089,-0.139008,0.001103,0.239862,2.843670,1653.395453,0.018781
2,2825.0,29.0,put,0.105266,2806.56405,-1.801200,3.778041,-0.390646,0.003959,-0.241137,46.445181,805.489634,0.009150
3,2825.0,29.0,put,0.105266,2980.16595,-0.199727,1.162341,-0.136470,0.001080,0.242650,2.644140,1633.442492,0.018554
